# DB2-Salesforce connector: Determine cluster organization

In [1]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'nanoHUB_user_ID__c'
object_id = 'Contact'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username": "wang159-4j1v@force.com",
    "password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",
} #6Cel800D5w000001jiQu8885w0000003U7mRvSj2dp1TvmgHngvv195i9eJLpzFNuMZBHkjibu07pj7Mk18p7MQxBEO318UtbzlBv7BpFQU

In [2]:
import sys
#sys.path.append(lib_dir)
    
import pandas as pd
import datetime

## Obtain contacts from DB2 
that have last visit date within range of interest specified by day_range

In [3]:
# Depending on the task, use different query and cutoff

sql_query = "select id, name, username, block, email, sendEmail, registerDate, lastvisitDate \
                    from jos_users"

# display
print(sql_query)

select id, name, username, block, email, sendEmail, registerDate, lastvisitDate                     from jos_users


In [4]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %(sql_login_params['username'], sql_login_params['password']))
df = pd.read_sql_query(sql_query, engine)

# get user profile details
profile_df = pd.read_sql_query("select * from jos_user_profiles where profile_key in ('orgtype', 'organization', 'orcid')", engine)

In [5]:
# display
display(df.head(1))

display(profile_df.head(1))

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate
0,998,hubrepo hubrepo,hubrepo,0,nkissebe@gmail.com,0,2014-11-13 21:09:09,NaT


,id,user_id,profile_key,profile_value,ordering,access
0,1,15623,orgtype,university,2,5


In [6]:
df = pd.merge(df, profile_df[profile_df['profile_key'] == 'orgtype'][['user_id', 'profile_value']],\
                         how='left', left_on='id', right_on='user_id')\
    .drop(columns=['user_id']).rename(columns={'profile_value':'orgtype'})

df = pd.merge(df, profile_df[profile_df['profile_key'] == 'organization'][['user_id', 'profile_value']],\
                         how='left', left_on='id', right_on='user_id')\
    .drop(columns=['user_id']).rename(columns={'profile_value':'organization'})

df = pd.merge(df, profile_df[profile_df['profile_key'] == 'orcid'][['user_id', 'profile_value']],\
                         how='left', left_on='id', right_on='user_id')\
    .drop(columns=['user_id']).rename(columns={'profile_value':'orcid'})

# display
display(df.head(1))

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid
0,998,hubrepo hubrepo,hubrepo,0,nkissebe@gmail.com,0,2014-11-13 21:09:09,NaT,NaN,NaN,NaN


In [7]:
# Obtain company domain information from DB2
domain_df = pd.read_sql_query("select name as domain_name, domain, industry, `size range` as size, country \
from wang159_myrmekes.companies_email_domain", engine)

In [8]:
# make sure domain is unique and drop NaN
domain_df = domain_df.drop_duplicates(subset='domain', keep='first').dropna()

In [9]:
# get email domain of nanoHUB users
def get_domain(this_email):
    seg_list = this_email.split('@')
    
    if len(seg_list) == 2:
        return seg_list[1].lower()
    else:
        return None
    
df['email_domain'] = df['email'].apply(get_domain)

In [10]:
df = pd.merge(df, domain_df[domain_df.domain.isin(df['email_domain'].unique()) & ~domain_df.domain.isna()],\
         how='left', left_on='email_domain', right_on='domain')

# display
display(df.head(2))

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_domain,domain_name,domain,industry,size,country
0,998,hubrepo hubrepo,hubrepo,0,nkissebe@gmail.com,0,2014-11-13 21:09:09,NaT,NaN,NaN,NaN,gmail.com,NaN,NaN,NaN,NaN,NaN
1,1000,CMS Admin Manager,admin,0,support@nanohub.org,0,NaT,NaT,NaN,NaN,NaN,nanohub.org,NaN,NaN,NaN,NaN,NaN


## Obtain newly updated organization IDs from Salesforce

In [11]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [12]:
# get Salesforce ID for organizations
sf_cluster_df = db_s.query_data('SELECT Id, Contact__c, Tool_Usage_Cluster__c FROM ContactToolClusterAssociation__c')

# display
sf_cluster_df.head(3)

[Success] Bulk job creation successful. Job ID = 7505w00000Mt5ksAAB
{"id":"7505w00000Mt5ksAAB","operation":"query","object":"ContactToolClusterAssociation__c","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-06-09T02:49:21.000+0000","systemModstamp":"2020-06-09T02:49:21.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000Mt5ksAAB","operation":"query","object":"ContactToolClusterAssociation__c","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-06-09T02:49:21.000+0000","systemModstamp":"2020-06-09T02:49:26.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":84982,"retries":0,"totalProcessingTime":5122}
[Success] Bulk job completed successfully.


,Contact__c,Id,Tool_Usage_Cluster__c
0,0035w000034JtebAAC,a0x5w00000ZtjVpAAJ,a0w5w000009Q7DnAAK
1,0035w000034I5Y6AAK,a0x5w00000ZtjVqAAJ,a0w5w000009Q8G8AAK
2,0035w000034JqhvAAC,a0x5w00000ZtjVrAAJ,a0w5w000009Q8G8AAK


In [13]:
# get Contacts from Salesforce
sf_contact_df = db_s.query_data('SELECT Id, Organization__c, Organization_email_derived__c, Organization_Overwrite__c FROM Contact')

# display
sf_contact_df.head(3)

[Success] Bulk job creation successful. Job ID = 7505w00000Mt5kxAAB
{"id":"7505w00000Mt5kxAAB","operation":"query","object":"Contact","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-06-09T02:49:36.000+0000","systemModstamp":"2020-06-09T02:49:36.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000Mt5kxAAB","operation":"query","object":"Contact","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-06-09T02:49:36.000+0000","systemModstamp":"2020-06-09T02:49:36.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000Mt5kxAAB","operation":"query","object":"Contact","createdById":"0055w00000ArpYvAAJ","create

,Id,Organization_Overwrite__c,Organization__c,Organization_email_derived__c
0,0035w000031Vsp1AAC,NaN,NaN,NaN
1,0035w000031Vsp2AAC,NaN,purdue university,a0r5w00000V42cCAAR
2,0035w000031Vsp3AAC,NaN,purdue university,a0r5w00000V42cCAAR


In [14]:
# get organization from Salesforce
sf_org_df = db_s.query_data('SELECT Id, Name, Domain__c FROM organization__c')

# display
sf_org_df.head(3)

[Success] Bulk job creation successful. Job ID = 7505w00000Mt5l2AAB
{"id":"7505w00000Mt5l2AAB","operation":"query","object":"organization__c","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-06-09T02:50:02.000+0000","systemModstamp":"2020-06-09T02:50:02.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000Mt5l2AAB","operation":"query","object":"organization__c","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-06-09T02:50:02.000+0000","systemModstamp":"2020-06-09T02:50:04.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":6173,"retries":0,"totalProcessingTime":528}
[Success] Bulk job completed successfully.


,Domain__c,Id,Name
0,upenn.edu,a0r5w00000V42c0AAB,University of Pennsylvania
1,fer.hr,a0r5w00000V42c1AAB,fer
2,wheatoncollege.edu,a0r5w00000V42c2AAB,Wheaton College


In [15]:
# get organization according to citation from Salesforce
sf_cit_org_df = db_s.query_data('SELECT Id, Contact__c, Organization__c FROM contact_citation_asso__c')

# display
sf_cit_org_df.head(3)

[Success] Bulk job creation successful. Job ID = 7505w00000Mt5lCAAR
{"id":"7505w00000Mt5lCAAR","operation":"query","object":"contact_citation_asso__c","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-06-09T02:50:15.000+0000","systemModstamp":"2020-06-09T02:50:15.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000Mt5lCAAR","operation":"query","object":"contact_citation_asso__c","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-06-09T02:50:15.000+0000","systemModstamp":"2020-06-09T02:50:16.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":4885,"retries":0,"totalProcessingTime":463}
[Success] Bulk job completed successfully.


,Contact__c,Id,Organization__c
0,0035w000034JLpOAAW,a0u5w00000RhVhTAAV,a0r5w00000V42cCAAR
1,0035w000034JLpOAAW,a0u5w00000RhVhUAAV,a0r5w00000V42cCAAR
2,0035w000034JLpOAAW,a0u5w00000RhVhVAAV,a0r5w00000V42cCAAR


In [16]:
# rename Id of each Salesforce DF to avoid collision
sf_cluster_df.rename(columns={'Id':'cluster_Id'}, inplace=True)
sf_org_df.rename(columns={'Id':'org_Id'}, inplace=True)
sf_contact_df.rename(columns={'Id':'contact_Id'}, inplace=True)
sf_cit_org_df.rename(columns={'Id':'cit_org_Id'}, inplace=True)

## Determine the most likely organization of each cluster

In [17]:
# data given
display('sf_cluster_df'); display(sf_cluster_df.head(1))
display('sf_org_df'); display(sf_org_df.head(1))
display('sf_contact_df'); display(sf_contact_df.head(1))
display('sf_cit_org_df'); display(sf_cit_org_df.head(1))

'sf_cluster_df'

,Contact__c,cluster_Id,Tool_Usage_Cluster__c
0,0035w000034JtebAAC,a0x5w00000ZtjVpAAJ,a0w5w000009Q7DnAAK


'sf_org_df'

,Domain__c,org_Id,Name
0,upenn.edu,a0r5w00000V42c0AAB,University of Pennsylvania


'sf_contact_df'

,contact_Id,Organization_Overwrite__c,Organization__c,Organization_email_derived__c
0,0035w000031Vsp1AAC,NaN,NaN,NaN


'sf_cit_org_df'

,Contact__c,cit_org_Id,Organization__c
0,0035w000034JLpOAAW,a0u5w00000RhVhTAAV,a0r5w00000V42cCAAR


In [18]:
# merge cluster with contact info
cc_df = pd.merge(sf_cluster_df, sf_contact_df, how='left', left_on='Contact__c', right_on='contact_Id')

# display
display(cc_df.head(3))

,Contact__c,cluster_Id,Tool_Usage_Cluster__c,contact_Id,Organization_Overwrite__c,Organization__c,Organization_email_derived__c
0,0035w000034JtebAAC,a0x5w00000ZtjVpAAJ,a0w5w000009Q7DnAAK,0035w000034JtebAAC,NaN,NaN,a0r5w00000V42jBAAR
1,0035w000034I5Y6AAK,a0x5w00000ZtjVqAAJ,a0w5w000009Q8G8AAK,0035w000034I5Y6AAK,NaN,university of southern california,a0r5w00000V42jBAAR
2,0035w000034JqhvAAC,a0x5w00000ZtjVrAAJ,a0w5w000009Q8G8AAK,0035w000034JqhvAAC,NaN,university of southern california,a0r5w00000V42jBAAR


In [19]:
# apply overwrite
cc_df['org_final'] = cc_df.fillna('').apply(lambda x: \
                    x['Organization_email_derived__c'] if x['Organization_Overwrite__c']=='' else x['Organization_Overwrite__c'],\
                                 axis=1)

In [20]:
# find the most likely org within each cluster
def get_likely_org(this_cluster_df):
    non_empty_df = this_cluster_df[this_cluster_df.org_final != '']
    
    if non_empty_df.shape[0] > 0:
        # there are non-empty organization information
        return non_empty_df.org_final.mode().iloc[0]
    else:
        return ''

        
cluster_org_df = cc_df[['Tool_Usage_Cluster__c', 'org_final']].groupby('Tool_Usage_Cluster__c')\
                            .apply(get_likely_org).reset_index(name='Organization_email_derived__c')

# display
cluster_org_df.head(3)

,Tool_Usage_Cluster__c,Organization_email_derived__c
0,a0w5w000009Q7CqAAK,a0r5w00000V42cCAAR
1,a0w5w000009Q7CrAAK,a0r5w00000V42cCAAR
2,a0w5w000009Q7CsAAK,a0r5w00000V42cCAAR


## Write to Salesforce

In [21]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf['Id'] = cluster_org_df['Tool_Usage_Cluster__c']

# derived information
df_sf['Organization__c'] = cluster_org_df['Organization_email_derived__c']

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Id,a0w5w000009Q7CqAAK,a0w5w000009Q7CrAAK
Organization__c,a0r5w00000V42cCAAR,a0r5w00000V42cCAAR


In [22]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

db_s.object_id = 'tool_usage_cluster__c'
db_s.external_id = 'Id'

Obtained Salesforce access token ...... True


In [23]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000Mt5lMAAR
hello
[Success] CSV upload successful. Job ID = 7505w00000Mt5lMAAR
[Success] Closing job successful. Job ID = 7505w00000Mt5lMAAR


In [24]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000Mt5lMAAR',
 'operation': 'upsert',
 'object': 'tool_usage_cluster__c',
 'createdById': '0055w00000ArpYvAAJ',
 'createdDate': '2020-06-09T02:50:35.000+0000',
 'systemModstamp': '2020-06-09T02:50:37.000+0000',
 'state': 'InProgress',
 'externalIdFieldName': 'Id',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

## Determine the most likely organization of each contact

### Rules

In the following order:

- If overwrite set, use overwrite
- If citation exists, use citation
- If email exists, use email
- If cluster exists, use cluster
- If none above, none

In [25]:
# data given
display('sf_cluster_df'); display(sf_cluster_df.head(1))
display('sf_org_df'); display(sf_org_df.head(1))
display('sf_contact_df'); display(sf_contact_df.head(1))
display('sf_cit_org_df'); display(sf_cit_org_df.head(1))
display('cluster_org_df'); display(cluster_org_df.head(1))

'sf_cluster_df'

,Contact__c,cluster_Id,Tool_Usage_Cluster__c
0,0035w000034JtebAAC,a0x5w00000ZtjVpAAJ,a0w5w000009Q7DnAAK


'sf_org_df'

,Domain__c,org_Id,Name
0,upenn.edu,a0r5w00000V42c0AAB,University of Pennsylvania


'sf_contact_df'

,contact_Id,Organization_Overwrite__c,Organization__c,Organization_email_derived__c
0,0035w000031Vsp1AAC,NaN,NaN,NaN


'sf_cit_org_df'

,Contact__c,cit_org_Id,Organization__c
0,0035w000034JLpOAAW,a0u5w00000RhVhTAAV,a0r5w00000V42cCAAR


'cluster_org_df'

,Tool_Usage_Cluster__c,Organization_email_derived__c
0,a0w5w000009Q7CqAAK,a0r5w00000V42cCAAR


In [26]:
display(sf_org_df.head(2))

,Domain__c,org_Id,Name
0,upenn.edu,a0r5w00000V42c0AAB,University of Pennsylvania
1,fer.hr,a0r5w00000V42c1AAB,fer


In [27]:
# build domain, org cluster ID as salesforce cluster
sf_org_ID_df = pd.DataFrame()
sf_org_ID_df['Domain__c'] = sf_org_df.iloc[:,0]
sf_org_ID_df['Salesforce_ID'] = sf_org_df.iloc[:,1]
display(sf_org_ID_df.head(2))


,Domain__c,Salesforce_ID
0,upenn.edu,a0r5w00000V42c0AAB
1,fer.hr,a0r5w00000V42c1AAB


In [28]:
# join salesforce ID back to contact DF
df = pd.merge(sf_org_ID_df, df, how='right', left_on='Domain__c', right_on='domain').drop('Domain__c', axis=1)\
            .rename(columns={'Id':'Salesforce_ID'})

In [33]:
# display
display(df.head(2))

,Salesforce_ID,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_domain,domain_name,domain,industry,size,country
0,a0r5w00000V42c0AAB,128134,"Yu, Chendi",chendiyu,0,chendiyu@upenn.edu,-1,2015-09-03 00:33:27,2016-07-04 14:29:33,NaN,NaN,NaN,upenn.edu,university of pennsylvania,upenn.edu,higher education,10001+,united states
1,a0r5w00000V42c0AAB,128913,Kameron Leo Oser,kamoser,0,kamoser@upenn.edu,-1,2015-09-13 02:38:17,2015-09-17 18:09:11,NaN,NaN,NaN,upenn.edu,university of pennsylvania,upenn.edu,higher education,10001+,united states


## Match data with Salesforce format

In [31]:
# split full name into first, middle, and last names
def split_full_name(this_name):
    this_name_list = list(filter(None, this_name.split(' ')))
    
    if len(this_name_list) == 1:
        # single word name
        return pd.Series([this_name_list[0],None, this_name_list[0]])
    
    elif len(this_name_list) > 1:
        # multi word name
        return pd.Series([this_name_list[0],' '.join(this_name_list[1:-1]),this_name_list[-1]])

In [32]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf[['firstname', 'Middle_name__c', 'lastname']] = df['name'].apply(split_full_name)

df_sf['nanoHUB_user_ID__c']            = df['id']
df_sf['nanoHUB_username__c']           = df['username']
df_sf['Email']                         = df['email'].fillna('').apply(lambda x: '' if '@invalid' in x else x)

# for sendEmail: 0 = opt-out, 1 = receive email. For salesforce HasOptedOutOfEmail, it's exact opposite
df_sf['HasOptedOutOfEmail']            = df['sendEmail'].apply(lambda x: 0 if x==1 else 1)
df_sf['nanoHUB_account_BLOCKED__c']    = df['block'].fillna(0)

# solidify time-related columns from datetime to string
df_sf['nanoHUB_registration_date__c']  = df['registerDate'].dt.date.fillna('').astype('str')
df_sf['nanoHUB_last_active_date__c']   = df['lastvisitDate'].dt.date.fillna('').astype('str')

# Tableau detailed view
df_sf['Detailed_user_timeline_to_Tableau__c'] = df_sf['Email'].apply(lambda x: 'https://tableauqa.itap.purdue.edu/views/profile/ProfileTimeline?Id%20Email='+x+'\
&:iframeSizedToWindow=true&:embed=y&:showAppBanner=false\
&:display_count=no&:showVizHome=no#6' if x != '' else '')

# nanoHUB user profile
df_sf['nanoHUB_user_page__c'] = df_sf['nanoHUB_user_ID__c'].apply(lambda x: 'https://nanohub.org/members/%d'%x if x != '' else '')

df_sf['ORCID__c'] = df['orcid'].fillna('')
df_sf['Organization__c'] = df['organization'].fillna('')
df_sf['Organization_Type__c'] = df['orgtype'].fillna('')

# derived information
df_sf['Organization_email_derived__c'] = df['Salesforce_ID']

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
firstname,"Yu,",Kameron
Middle_name__c,,Leo
lastname,Chendi,Oser
nanoHUB_user_ID__c,128134,128913
nanoHUB_username__c,chendiyu,kamoser
Email,chendiyu@upenn.edu,kamoser@upenn.edu
HasOptedOutOfEmail,1,1
nanoHUB_account_BLOCKED__c,0,0
nanoHUB_registration_date__c,2015-09-03,2015-09-13
nanoHUB_last_active_date__c,2016-07-04,2015-09-17


## To Salesforce Sales Cloud CRM

In [34]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

db_s.object_id = object_id
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [41]:
## testing yahoo removal
df_sf['Organization__c'] = df_sf['Organization__c'].replace(to_replace='yahoo',value='   ')
display(df_sf.head(5))

,firstname,Middle_name__c,lastname,nanoHUB_user_ID__c,nanoHUB_username__c,Email,HasOptedOutOfEmail,nanoHUB_account_BLOCKED__c,nanoHUB_registration_date__c,nanoHUB_last_active_date__c,Detailed_user_timeline_to_Tableau__c,nanoHUB_user_page__c,ORCID__c,Organization__c,Organization_Type__c,Organization_email_derived__c
0,"Yu,",,Chendi,128134,chendiyu,chendiyu@upenn.edu,1,0,2015-09-03,2016-07-04,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/128134,,,,a0r5w00000V42c0AAB
1,Kameron,Leo,Oser,128913,kamoser,kamoser@upenn.edu,1,0,2015-09-13,2015-09-17,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/128913,,,,a0r5w00000V42c0AAB
2,Akshay,,Ananthakrishnan,131613,akshay91,akshayan@upenn.edu,1,0,2015-10-19,2015-10-19,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/131613,,University of Pennsylvania,universitygraduate,a0r5w00000V42c0AAB
3,"Azadi,",,Mohsen,136731,azadi,azadi@upenn.edu,1,0,2015-12-28,2015-12-28,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/136731,,,,a0r5w00000V42c0AAB
4,"Zhou,",,Songsong,139502,songzhou,songzhou@upenn.edu,1,0,2016-02-10,2016-02-10,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/139502,,,,a0r5w00000V42c0AAB


In [42]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000Mt5utAAB
hello
[Success] CSV upload successful. Job ID = 7505w00000Mt5utAAB
[Success] Closing job successful. Job ID = 7505w00000Mt5utAAB


In [43]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000Mt5utAAB',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000ArpYvAAJ',
 'createdDate': '2020-06-09T03:08:49.000+0000',
 'systemModstamp': '2020-06-09T03:10:29.000+0000',
 'state': 'InProgress',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [44]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

''
